<a href="https://colab.research.google.com/github/JuanBarros2/tcc_andromedev/blob/main/Pre_Formating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os 
import pandas as pd
import numpy as np
import random
import re
import time

from google.colab import drive
drive.mount('/content/drive')

path =  'drive/MyDrive/TCC_DATA/data/'

Mounted at /content/drive


In [ ]:
inscritos = pd.read_csv(path + "Inscritos.csv")
mentoria = pd.read_csv(path + "Mentoria.csv")
organizacoes = pd.read_csv(path + "Organizacoes.csv")
interessados = pd.read_csv(path + "InteressadosPrevios.csv")
overview = pd.read_csv(path + "Overview.csv")

ep1 = pd.read_csv(path + "EP1.csv")
ep2 = pd.read_csv(path + "EP2.csv")

ef_mentoria = pd.read_csv(path + "EF_Mentoria.csv")
ef_aprendiz = pd.read_csv(path + "EF_Aprendiz.csv")

fs_aprendiz = pd.read_csv(path + "FS_Aprendiz.csv")
fs_mentoria = pd.read_csv(path + "FS_Mentoria.csv")

In [ ]:
def blank_id_df(df, fields, rename_id, remove_fields):
    random_id_df = list(range(0, df.shape[0]))
    random.shuffle(random_id_df)
    df[rename_id] = random_id_df
    result = df[[*fields, rename_id]]
    df = df.drop(columns=[*remove_fields, *fields]).sort_values(rename_id)
    
    return (df, result)

def replace_column(df, relationship, rename_column, remove_column, id_label):
    update_relationship = relationship.rename({ rename_column[0]: rename_column[1]}, axis = 1).drop(columns=remove_column)
    
    result = df.merge(update_relationship, on=[rename_column[1]], how='left').drop(columns=rename_column[1])
    result = result.rename({ id_label: rename_column[1]}, axis = 1)
    
    return result

In [ ]:
inscritos, ids_inscritos = blank_id_df(inscritos, ['email', 'nome'], 'id_inscrito', ['discord_id','github_link'])
mentoria, ids_mentorias = blank_id_df(mentoria,  ['email', 'nome'], 'id_mentoria', ['discord_id'])
organizacoes, ids_organizacoes = blank_id_df(organizacoes, ['nome'], 'id_organizacao', ['email']) 
_, ids_projetos = blank_id_df(overview, ['projeto'], 'id_projeto', []) 

In [ ]:
overviewPRE = replace_column(overview, ids_inscritos, ('nome', 'aprendiz'), ['email'], 'id_inscrito')
overviewPRE = replace_column(overviewPRE, ids_mentorias, ('nome', 'mentor_principal'), ['email'], 'id_mentoria')
overviewPRE = replace_column(overviewPRE, ids_mentorias, ('nome', 'mentor_auxiliar'), ['email'], 'id_mentoria')
overviewPRE = overviewPRE.drop(columns=['projeto', 'organizacao'])
overviewPRE[overviewPRE['mentor_principal'].notna()].shape

(50, 10)

In [ ]:
interessadosPRE = interessados.merge(ids_inscritos, on=['email'], how='left' )
interessadosPRE = interessadosPRE.drop(columns=['email', 'nome'])


In [ ]:
ep1PRE = ep1.merge(ids_projetos, on=['projeto'], how='right' )
ep1PRE = ep1PRE.drop(columns=["email", "projeto"])
ep1PRE = ep1PRE.rename({ 'id_projeto':  'projeto' }, axis = 1)
ep1PRE = ep1PRE.drop_duplicates(subset=['projeto'], keep='last') 

ep2PRE = ep2.merge(ids_projetos, on=['projeto'], how='right')
ep2PRE = ep2PRE.drop(columns=["email", "projeto"])
ep2PRE = ep2PRE.rename({ 'id_projeto':  'projeto' }, axis = 1)
ep2PRE = ep2PRE.drop_duplicates(subset=['projeto'], keep='last') 

In [ ]:
ep1PRE[ep1PRE['projeto'].isna()]

,data,Q1,Q2,Q3,projeto


In [ ]:
ep2PRE.groupby('projeto').size()

In [ ]:
ef_aprendiz['projeto'] = ef_aprendiz['projeto'].map(lambda projeto: re.sub(r"\[.*\] ", "", projeto))
ef_aprendizPRE = ef_aprendiz.drop_duplicates(subset=['projeto'],keep='last')
ef_aprendizPRE = ef_aprendizPRE.merge(ids_projetos, on=['projeto'], how='right')
ef_aprendizPRE = ef_aprendizPRE.drop(columns=["email", "projeto"])
ef_aprendizPRE = ef_aprendizPRE.rename({ 'id_projeto':  'projeto'}, axis = 1)
ef_aprendizPRE = ef_aprendizPRE.drop(columns=["Q8"])

ef_mentoria['projeto'] = ef_mentoria['projeto'].map(lambda projeto: re.sub(r"\[.*\] ", "", projeto))
ef_mentoriaPRE = ef_mentoria.drop_duplicates(subset=['projeto'],keep='last')
ef_mentoriaPRE = ef_mentoriaPRE.merge(ids_projetos, on=['projeto'], how='right')
ef_mentoriaPRE = ef_mentoriaPRE.drop(columns=["email", "projeto"])
ef_mentoriaPRE = ef_mentoriaPRE.rename({ 'id_projeto':  'projeto' }, axis = 1)
ef_mentoriaPRE.shape

(50, 14)

In [ ]:
ef_aprendizPRE[ef_aprendizPRE['data'].notna()].shape

(41, 18)

In [ ]:
aprendizes = overviewPRE[["id_projeto","aprendiz"]]
aprendizes = aprendizes.rename(columns={'aprendiz': 'id_inscrito'})

In [ ]:
fs_aprendizPRE = fs_aprendiz.merge(ids_inscritos, on=['email'])
fs_aprendizPRE = fs_aprendizPRE.drop(columns=["email", "projeto", "nome"])
fs_aprendizPRE = fs_aprendizPRE.drop_duplicates(subset=['id_inscrito', 'semana'],keep='last') 


In [ ]:
semanas = list(dict.fromkeys(fs_aprendizPRE['semana']) )
def apply_atraso(row):
  data_limite = ['27/09/2020', '04/10/2020', '11/10/2020', '18/10/2020', '25/10/2020', '01/11/2020', '08/11/2020', '15/11/2020']
  semana = semanas.index(row['semana'])
  data = time.strptime(row['data'][:10] if type(row['data']) is str else '16/11/2020', "%d/%m/%Y")
  row['atraso'] = (data > time.strptime(data_limite[semana], "%d/%m/%Y") )
  return row

fs_aprendizPRE = fs_aprendizPRE.apply(apply_atraso, axis=1)

aux = pd.DataFrame(index = pd.MultiIndex.from_product([aprendizes['id_inscrito'], semanas], names = ["id_inscrito", "semana"])).reset_index()
fs_aprendizPRE = fs_aprendizPRE.merge(aux, on=['semana', 'id_inscrito'], how="right")
fs_aprendizPRE.groupby('id_inscrito').size().reset_index(name='count')



,id_inscrito,count
0,3,8
1,5,8
2,11,8
3,17,8
4,19,8
5,22,8
6,23,8
7,26,8
8,30,8
9,34,8


In [ ]:
overviewPRE[overviewPRE['aprendiz'] ==73]


,FS,EP1,EP2,EF,apresentacao,areas,id_projeto,aprendiz,mentor_principal,mentor_auxiliar
28,True,True,True,True,True,Implementação de features,20,73,54,10


In [ ]:

ids_inscritos[ids_inscritos['id_inscrito'] ==73]

,email,nome,id_inscrito
26,dacio.bezerra@ccc.ufcg.edu.br,Dacio Silva Bezerra,73


In [ ]:
fs_mentoriaPRE = fs_mentoria.merge(ids_mentorias, on=['email'], how='left')
fs_mentoriaPRE = fs_mentoriaPRE.drop(columns=["email", "nome"])
fs_mentoriaPRE = fs_mentoriaPRE.merge(ids_projetos, on=['projeto'], how='left')
fs_mentoriaPRE = fs_mentoriaPRE.drop_duplicates(subset=['id_projeto', 'semana'],keep='last') 
fs_mentoriaPRE = fs_mentoriaPRE.apply(apply_atraso, axis=1)

aux = pd.DataFrame(index = pd.MultiIndex.from_product([ids_projetos['id_projeto'], semanas], names = ["id_projeto", "semana"])).reset_index()
fs_mentoriaPRE = fs_mentoriaPRE.merge(aux, on=['semana', 'id_projeto'], how="right")
fs_mentoriaPRE[fs_mentoriaPRE['id_projeto'].isna()]

,data,semana,projeto,Q1,Q2,Q3,Q4,Q5,id_mentoria,id_projeto,atraso


In [ ]:
areasPRE = overviewPRE.assign(areas=overviewPRE.areas.str.split(", ")).explode('areas')
areasPRE = areasPRE[['id_projeto','areas']]


In [ ]:
overviewPRE.to_csv(path + 'pre/OverviewPRE.csv', index=False)
ids_projetos.to_csv(path + 'pre/ProjetosPRE.csv', index=False)
organizacoes.to_csv(path + 'pre/OrganizacoesPRE.csv', index=False)

inscritos.to_csv(path + 'pre/InscritoPRE.csv', index=False)
mentoria.to_csv(path + 'pre/MentoriaPRE.csv', index=False)
interessadosPRE.to_csv(path + 'pre/InteressadosPreviosPRE.csv', index=False)
ep1PRE.to_csv(path + 'pre/EP1PRE.csv', index=False)
ep2PRE.to_csv(path + 'pre/EP2PRE.csv', index=False)
areasPRE.to_csv(path + 'pre/AreasPRE.csv', index=False)
ef_mentoriaPRE.to_csv(path + 'pre/EF_MentoriaPRE.csv', index=False)
ef_aprendizPRE.to_csv(path + 'pre/EF_AprendizPRE.csv', index=False)
fs_mentoriaPRE.to_csv(path + 'pre/FS_MentoriaPRE.csv', index=False)
fs_aprendizPRE.to_csv(path + 'pre/FS_AprendizPRE.csv', index=False)
aprendizes.to_csv(path + 'pre/AprendizPRE.csv', index=False)